1.이미지 url 파싱

2.이미지 feature 추출(ResNet50)

3.이미지 변형 지표(image_deviation) 만들기 (가설 검정용 핵심 변수)-PCA 첫 번째 성분(PC1) 기반 변형지표

4.위험요인 vs 이미지 변형 검정 (가설 검정)

(1) 가격 vs 이미지 변형 (Mann–Whitney U test)

(2) 건물유형 vs 이미지 변형 (Kruskal–Wallis)

(3) 연식(built_year) vs 이미지 변형

In [1]:
# =============================
# 0. 라이브러리 설치
# =============================
!pip install -q tensorflow pillow scikit-learn tqdm

import pandas as pd
import numpy as np
from tqdm import tqdm
import requests
from io import BytesIO
from PIL import Image

import tensorflow as tf
from tensorflow.keras.applications import ResNet50, resnet50
from tensorflow.keras import layers, models

from sklearn.decomposition import PCA
from scipy.stats import mannwhitneyu, kruskal

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# =============================
# 1. CSV 로드
# =============================
csv_path = "/content/drive/MyDrive/daangn_list_detail.csv"  # 경로 수정 가능
df = pd.read_csv(csv_path)

print("데이터 Shape:", df.shape)
print(df.columns)

데이터 Shape: (1059, 16)
Index(['area', 'identifier', 'description', 'image_count', 'image',
       'building_name', 'building_usage', 'exclusive_area', 'floor',
       'direction', 'maintenance_fee', 'built_year', 'total_floor', 'price',
       'address', 'register_date'],
      dtype='object')


In [4]:
# =============================
# 2. 이미지 URL 파싱
# =============================

def parse_urls(x):
    if pd.isna(x):
        return []
    urls = [u.strip() for u in str(x).split("|") if len(u.strip()) > 5]
    return urls

df["image_list"] = df["image"].apply(parse_urls)

df["num_images_parsed"] = df["image_list"].apply(len)
print(df["num_images_parsed"].describe())

count    1059.000000
mean        7.403211
std         3.299600
min         0.000000
25%         5.000000
50%         7.000000
75%         9.000000
max        27.000000
Name: num_images_parsed, dtype: float64


In [5]:
# =============================
# 3. ResNet50 Feature Extractor 준비
# =============================

base = ResNet50(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
base.trainable = False

feature_model = models.Sequential([
    layers.Input(shape=(256, 256, 3)),
    layers.Lambda(lambda z: resnet50.preprocess_input(z * 255.0)),
    base,
    layers.GlobalAveragePooling2D()
])

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [6]:
# =============================
# 4. 이미지 로딩 함수
# =============================

def load_image_from_url(url):
    try:
        r = requests.get(url, timeout=5)
        r.raise_for_status()
        img = Image.open(BytesIO(r.content)).convert("RGB")
        img = img.resize((256, 256))
        img = np.array(img).astype("float32") / 255.0
        return img
    except:
        return None

In [7]:
# =============================
# 5. 매물별 이미지 Feature 추출
# =============================

feat_list = []
valid_idx = []

for idx, row in tqdm(df.iterrows(), total=len(df)):
    urls = row["image_list"]

    if len(urls) == 0:
        feat_list.append(None)
        continue

    img_feats = []

    for url in urls:
        img_arr = load_image_from_url(url)
        if img_arr is None:
            continue
        img_batch = np.expand_dims(img_arr, axis=0)
        feat = feature_model.predict(img_batch, verbose=0)[0]
        img_feats.append(feat)

    if len(img_feats) == 0:
        feat_list.append(None)
    else:
        feat_list.append(np.mean(img_feats, axis=0))  # 여러 이미지 → 평균 feature

df["img_feature"] = feat_list

100%|██████████| 1059/1059 [1:07:57<00:00,  3.85s/it]


In [8]:
# =============================
# 6. Feature None 제거 후 PCA 진행
# =============================

feat_rows = df["img_feature"].dropna()
feature_matrix = np.vstack(feat_rows.values)

pca = PCA(n_components=1)
pc1 = pca.fit_transform(feature_matrix).flatten()

df.loc[feat_rows.index, "img_pc1"] = pc1

In [9]:
# =============================
# 7. 이미지 변형 지표 (image_deviation)
# =============================

median_pc1 = df["img_pc1"].median()
df["image_deviation"] = abs(df["img_pc1"] - median_pc1)

print(df["image_deviation"].describe())

count    1053.000000
mean        4.837322
std         2.982992
min         0.000000
25%         2.417381
50%         4.551473
75%         6.840476
max        15.345619
Name: image_deviation, dtype: float64


In [32]:
# =============================
# 8. 위험요인 vs 이미지 변형 검정 (최종 안정 버전)
# =============================

from scipy.stats import mannwhitneyu, kruskal
import numpy as np

results = {}

# ==========================================================
# (1) 가격 vs 이미지 변형
# ==========================================================
price_thr = df["price"].quantile(0.2)  # 하위 20%
cheap = df[df["price"] <= price_thr]["image_deviation"].dropna()
normal = df[df["price"] > price_thr]["image_deviation"].dropna()

stat, p_price = mannwhitneyu(cheap, normal, alternative="two-sided")
results["price_vs_image"] = p_price


# ==========================================================
# (2) 건물유형 vs 이미지 변형 (NaN 완전 제거 버전)
# ==========================================================

# 1) 그룹별 리스트 만들기
groups_raw = df.groupby("building_usage")["image_deviation"].apply(list)

# 2) NaN 제거 + 값이 2개 이상 있는 그룹만 선택
groups_clean = []
valid_labels = []

for label, values in groups_raw.items():
    clean_vals = [v for v in values if not np.isnan(v)]
    if len(clean_vals) > 1:  # 최소 2개 필요
        groups_clean.append(clean_vals)
        valid_labels.append(label)

# 3) Kruskal 검정 실행
if len(groups_clean) >= 2:
    stat, p_usage = kruskal(*groups_clean)
else:
    p_usage = np.nan

results["usage_vs_image"] = p_usage


# ==========================================================
# (3) 연식 vs 이미지 변형
# ==========================================================

def safe_year(y):
    try:
        year = int(str(y).strip()[:4])
        return year
    except:
        return np.nan

df["year_numeric"] = df["built_year"].apply(safe_year)

# 신축 여부 (2015년 기준)
df["is_new"] = df["year_numeric"].fillna(0) >= 2015

new = df[df["is_new"] == True]["image_deviation"].dropna()
old = df[df["is_new"] == False]["image_deviation"].dropna()

stat, p_year = mannwhitneyu(new, old, alternative="two-sided")
results["year_vs_image"] = p_year

In [33]:
# =============================
# 9. 결과 출력
# =============================
print("\n📌 가설4 검정 결과 (p-value):")
for k, v in results.items():
    print(f"{k}: {v}")

print("\n🔥 p < 0.05 이면 가설4(이미지 변형 ↑ → 사기 위험 ↑) 통계적으로 지지됨")


📌 가설4 검정 결과 (p-value):
price_vs_image: 2.469162933277011e-07
usage_vs_image: 2.0725206441004928e-05
year_vs_image: 0.9427978872620548

🔥 p < 0.05 이면 가설4(이미지 변형 ↑ → 사기 위험 ↑) 통계적으로 지지됨


**유의함**